In [ ]:
import numpy as np
import network
import matplotlib.pyplot as plt
from scipy import stats

## Aufgabe 5
> Hypothese: $\xi \propto |p_c - p|^{-\nu}$ für $p < p_c$ und $p \rightarrow p_c$

> Vorgehen: Linearisierung durch doppelt-logarithmische Auftragung

In [ ]:
L = 1000  # size of simulated networks
num = 100  # number of tested probabilities
p_c = 0.593   # see ex 2

p_arr = np.linspace(0, p_c, num)    # array of simulated probabilities
ln_zeta = []
ln_dp = []
j = 0

for p in p_arr:

    # print simulation-progress
    print(str(j*100/num)+'%')
    j += 1

    net = network.Network(L, L, p)
    net.hoshen_kopelman()
    zeta = net.correlation_length()
    dp = abs(p - p_c)

    # exclude percolating systems (condition p < p_c) and special cases
    if dp > 0 and zeta > 0 and not net.is_perculating() :
        ln_zeta.append(np.log(zeta))
        ln_dp.append(np.log(dp))


# plotting and linear regression
ln_dp_interpolate = np.linspace(ln_dp[0], ln_dp[-1], num)
plt.scatter(ln_dp, ln_zeta, s=4, c='r', label='Simulation')
res = stats.linregress(ln_dp, ln_zeta)
plt.plot(ln_dp_interpolate, ln_dp_interpolate * res.slope + res.intercept, label=r'ln($\zeta$) = ('
        +str(round(res.slope, 2))+r'$ \pm '+str(round(res.stderr, 2))+r') \cdot \ln(|p-p_c|) + $('
        +str(round(res.intercept, 2))+r' $\pm $'+str(round(res.intercept_stderr, 2))+')', linewidth=2, color='black')

plt.legend()
plt.grid()
plt.xlabel(r'ln(|$p-p_c$|)')
plt.ylabel(r'ln($\zeta$)')
plt.savefig('testex5')

Die Berechnung der Korrelationslänge findet in der Network-Klasse mit folgender Methode statt:

In [ ]:
def correlation_length(self):
    if not self.analysed: self.hoshen_kopelman()    # check that network is analyzed
    avg_square_distance = 0
    second_moment = 0
    cluster_sizes = self.cluster_sizes()
    for s in cluster_sizes:
        n_s = cluster_sizes[s] / (self.width * self.height)     # probability that a random tile belongs to a cluster of size s
        avg_square_distance += 2 * self.r_s_squared(s) * n_s * (s ** 2)
        second_moment += (s ** 2) * n_s

    return 0 if second_moment == 0 else np.sqrt(avg_square_distance / second_moment)


Die Berechnung von $R_S^2 = \sum_{i, j} \frac{|\mathbf{r_i - r_j}|^2}{S^2}$ erfolgt mithilfe der folgenden Klassen-Methode:

In [ ]:
def r_s_squared(self, s):

    # for clusters of size 0 or 1 return 0
    if s <= 1.0:
        return 0

    self.fix_labels()
    labels = np.where(self.N == s)[0]

    # check existence of sufficient labels
    if len(labels) == 0:
        return 0

    r_squared = 0

    # loop over all clusters of size s
    for label in labels:

        n = np.copy(self.labeled_network)   # copy to not modify this network
        n[n != label] = 0                   # set all wich are not the cluster to 0
        n = n[~np.all(n == 0, axis=1)]      # delete columns with 0
        n = n[:, ~np.all(n == 0, axis=0)]   # delete rows with 0

        # loop over all coordinate-combinations
        for i in range(len(n)):
            for j in range(len(n[i])):
                if n[i][j] != 0:
                    for k in range(len(n)):
                        for l in range(len(n[i])):
                            if n[k][l] != 0:
                                r_squared += ((i - k)**2 + (j - l)**2)

    r_squared /= (2 * s * s)    # factor 2 takes care of redundant term in the sum
    r_squared /= len(labels)    # normalize by the number of investigated clusters
    return r_squared
